In [ ]:
import qiskit
import numpy as np
import qiskit_nature
from qiskit import Aer
import matplotlib.pyplot as plt
import qiskit_nature.algorithms
from qiskit.algorithms import VQE
from ibm_quantum_widgets import *
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit_aer import AerSimulator
from warnings import filterwarnings
from qiskit.circuit import Parameter
from qiskit_nature.drivers import Molecule
from qiskit_nature.settings import settings
from qiskit import QuantumCircuit, transpile
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import EfficientSU2
import qiskit_nature.drivers.second_quantization
import qiskit_nature.problems.second_quantization
import qiskit_nature.transformers.second_quantization.electronic

filterwarnings('ignore')
settings.dict_aux_operators = False

In [ ]:
'''
Displays the diffrent entagelments types for the variational forms used in the 
vqe. The EfficientSU2() is a variational template that uses SU(2)gates for efficent
exploration of the multi-qubit Hilbert space. 
'''
entanglements = ["linear", "full"]
for entanglement in entanglements:
    form = EfficientSU2(num_qubits = 4, entanglement = entanglement)
    print(f"{entanglement} entanglement:")
    display(form.decompose().draw(fold = -1))

In [ ]:
 
#====================================================================#
# Prepares Qubit Operator For Quatum Simulations of Molecular System #
#====================================================================#

'''
Parameters: 
    - molecule: represents molecule info (geometry, multiplicity, charge)
    - remove_orbitals: specifies which orbitals to be removed in freeze core approx
Return:
    - qubit_op: qubit operator that encodes the lectronic Hamiltonian of the molecular system
    - num_particles: number of electrons in molecular system
    - num_spin_orbitals: number of spin orbitals in molecular system
    - problem: electrons structure problem of the molecule (includes info about Hamiltonian)
    - converter: converts second quantized Hamiltonian to qubit operator for quatum computation
'''
def get_qubit_op(molecule, remove_orbitals): 

    '''
    driver: translates molecular info into compatible formate
    molecule: defined in main loop
    basis: specifies set used in eletronic structre cacualtion. it describes wavefunctions
           of electrons in molecule. sto3g is represents Slater-type orbitals with 3 
           Gaussian funciton which is common choice for small molecules (good balance)
    driver_type: indicates the type of electronic structuer driver to be used. It's responsible 
                 for 1 & 2 electron intergrals which is is used for constructing electoic Hamiltonian
    '''
    driver = qiskit_nature.drivers.second_quantization.ElectronicStructureMoleculeDriver(
        molecule = molecule,
        basis = "sto3g",
        driver_type = qiskit_nature.drivers.second_quantization.ElectronicStructureDriverType.PYSCF)
    
    '''
    problem: encapsulate electronic structure problem which holds the relevant info to solve
             its moleculer structure and the Hamiltonian
    driver: gets info from driver
    remove_orbitlas: list of orbital indicies to exclude from caculation (reduces redudance)
    '''
    problem = qiskit_nature.problems.second_quantization.ElectronicStructureProblem(
        driver,
        remove_orbitals)

    second_q_ops = problem.second_q_ops()           # gets 2nd quantized operators (fermonic creation & annihilation operators)
    num_spin_orbitals = problem.num_spin_orbitals   # gets number of spin orbitals (cosider both up & down electrons)
    num_particles = problem.num_particles           # total number of electrons in system
    hamiltonian = second_q_ops[0]                   # gets hamiltonian operator (contains systems totlal energy)

    '''
    mapper: repsonsble to convert fermionic, bosonic, vibrational and spin operators 
            to qubit operators (using pauli operators X,Y,Z). ParityMapper() minimuzes number
            of auxiliary qubits neeeded to represent eletronic structure. 
    '''
    mapper = qiskit_nature.mappers.second_quantization.ParityMapper()  

    '''
    converter: Creates a qubit convert object thats responsible for translating 
               the fermionic Hamiltonian qubit representation in respect to the mapper.
    reducer: Applies two qubit reduction exploits to reduce the qubit Hamiltonian 
             symmetries in the system (num_particles input importaant for reduction strategy)
    qubit_op: Applies the convert and reducer to the qubit operator to be used for the
              quatum simulation
    '''
    converter = qiskit_nature.converters.second_quantization.QubitConverter(mapper, two_qubit_reduction = True)
    reducer = qiskit.opflow.TwoQubitReduction(num_particles)
    qubit_op = converter.convert(hamiltonian)
    qubit_op = reducer.convert(qubit_op)

    return qubit_op, num_particles, num_spin_orbitals, problem, converter

In [ ]:
#==================================================================#
# Finds The Exact Energy Level of The Current Interatomic Distance #
#==================================================================#
'''
Parameters: 
    - problem: object that represents electronc structure problem to be solved
    - coverter: object that has translated the fermionic Hamiltonian into a qubit representation
Returns:
    - result: object with information about molecule (e.g. energy level, wavefunction, etc)
'''
def exact_solver(problem, converter):
    '''
    solver: instantiates a classical eigensolver based on NumPy's linear algebra routines.
            (based on diagonalizing the Hamiltonian matrix.)
    calc:   instance of the eigen solver
    result: solves the eigen value if the given problem
    '''
    solver = qiskit_nature.algorithms.NumPyMinimumEigensolverFactory()
    calc = qiskit_nature.algorithms.GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

In [ ]:
#=================================================#
# Graphs The Energy Levels Found At Each Distance #
#=================================================#
def graph_results(distances, exact_energies, vqe_energies):
    plt.title("Grond State Energy Levels of Lithium Hydride (LiH)")
    plt.plot(distances, vqe_energies, 'x', label = "VQE Energy")
    plt.plot(distances, exact_energies, label = "Exact Energy")
    plt.xlabel("Atomic Distance (Angstrom)")
    plt.ylabel("Energy")
    plt.legend()
    plt.show()

In [ ]:

def main():
    vqe_energies = []                                               # holds vqe energies at each distance
    exact_energies = []                                             # holds exact energies at each distance
    distances = np.arange(0.5, 4.25, 0.25)                          # from 0.5 t0 4.0 with step 0.2 angstroms
    optimizer = qiskit.algorithms.optimizers.SLSQP(maxiter = 5)     # SLSQP optimizer with max 5 iterations
    backend = qiskit.BasicAer.get_backend("statevector_simulator")  # quatum simulator machine

    #  Iterates Through Each Interatomic Distance #  
    for dist in distances:
        '''
        geometry: represents type of atom and its Cartesion coordinates [x, y, z]
        multiplicity: represents total spin angular momentum of molecule's electon. multiplicity of 1 
                    means all elections are paried resulting in a singlet state (no unpaired electrons). 
                    Larger calues idicate elctrons higher spin states
        charge: represents net charge. 0 indicates neutral (Equal number of protons and electrons)
        '''
        molecule = Molecule(                
            geometry = [
                ["Li", [0.0, 0.0, 0.0] ],
                ["H", [dist, 0.0, 0.0] ]
            ],
            multiplicity = 1,  # = 2*spin+1
            charge = 0,
        )

        (qubit_op, num_particles, num_spin_orbitals, problem, converter) = get_qubit_op(molecule,
            [qiskit_nature.transformers.second_quantization.electronic.FreezeCoreTransformer(
            freeze_core=True, remove_orbitals=[-3,-2])])

        result = exact_solver(problem, converter)
        exact_energies.append(result.total_energies[0].real)

        '''
        init_state: Creates a good starting point for VQE which is a reasonable approximation of the true 
                    ground state for the current distance. This code returns a quantum circut that prepares
                    qubits in states that does the approximation
        '''
        init_state = qiskit_nature.circuit.library.HartreeFock(num_spin_orbitals, num_particles, converter)
        '''
        var_form: constucts the variational form for quantum simulation. It uses the UCCSD (Unitary Coupled
                  Cluster Singles and Doubles) ansatz. It starts with init_state then procseed to apply
                  single & two-qubit gates to create entanglement and approx ground state.
        '''
        var_form = qiskit_nature.circuit.library.UCCSD(converter,
                        num_particles,
                        num_spin_orbitals,
                        initial_state = init_state)
        
        # applies vqe algorthm & finds the min eigenvalue which is grondstate energy
        vqe = VQE(var_form, optimizer, quantum_instance = backend)
        vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
        vqe_result = problem.interpret(vqe_calc).total_energies[0].real
        vqe_energies.append(vqe_result)

        print(f"Interatomic Distance: {np.round(dist, 2)}  ",
              f"VQE Result: {vqe_result:.5f}  ",
              f"Exact Energy: {exact_energies[-1]:.5f}")
        
    print("All energies have been calculated")
    graph_results(distances, exact_energies, vqe_energies)

if __name__ == "__main__":
    main()